In [1]:
!python -V

Python 3.10.11


In [2]:
import pandas as pd
import numpy as np

import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/nildefonso/Zoomcamps/git/mlops-zoomcamp/week-2/mlruns/1', creation_time=1684925439711, experiment_id='1', last_update_time=1684925439711, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
#Create a function to read data and prepare everything
def read_dataframe(filename):
    # Read Data
    df = pd.read_parquet(filename)
    
    # Create new duration field in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    # Select the data which belongs to 98%/99% percentile
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df = df[(df.trip_distance >= 0) & (df.trip_distance <= 24)]
    
    # Define Categorical
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
dv = DictVectorizer()
categorical = ['PU_DO']
numerical = ['trip_distance']

# Create train dictonary and vectorize it
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Create validation dictonary and vectorize it
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
# Define target
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
#Train the model and predict
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

# Look into 
mean_squared_error(y_val, y_pred, squared=False)

6.6989660849932005

In [10]:
# Export the model built before
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [11]:
#Set experiment with MLflow
with mlflow.start_run():

    mlflow.set_tag("developer", "nuno")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [12]:
# Apply xgboost
import xgboost as xgb

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [34]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [35]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [17]:
params = {
'learning_rate': 0.2845520013920372,
'max_depth': 19,
'min_child_weight': 1.7059460308571863,
'objective': 'reg:linear',
'reg_alpha': 0.044418105842765995,
'reg_lambda': 0.04513790963019571,
'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2023/05/24 16:32:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6e2a0ecfccf14ce79737656a13f22786', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[16:32:55] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.00299
[1]	validation-rmse:12.65224
[2]	validation-rmse:10.42834
[3]	validation-rmse:8.99547
[4]	validation-rmse:8.08448
[5]	validation-rmse:7.52358
[6]	validation-rmse:7.17631
[7]	validation-rmse:6.96055
[8]	validation-rmse:6.82307
[9]	validation-rmse:6.73156
[10]	validation-rmse:6.67141
[11]	validation-rmse:6.62849
[12]	validation-rmse:6.60002
[13]	validation-rmse:6.57977
[14]	validation-rmse:6.56659
[15]	validation-rmse:6.55747
[16]	validation-rmse:6.55207
[17]	validation-rmse:6.54667
[18]	validation-rmse:6.54113
[19]	validation-rmse:6.53768
[20]	validation-rmse:6.53384
[21]	validation-rmse:6.52928
[22]	validation-rmse:6.52564
[23]	validation-rmse:6.52191
[24]	validation-rmse:6.51914
[25]	validation-rmse:6.51610
[26]	validation-rmse:6.51359
[27]	valid

[270]	validation-rmse:6.32852
[271]	validation-rmse:6.32804
[272]	validation-rmse:6.32810
[273]	validation-rmse:6.32767
[274]	validation-rmse:6.32740
[275]	validation-rmse:6.32770
[276]	validation-rmse:6.32766
[277]	validation-rmse:6.32698
[278]	validation-rmse:6.32668
[279]	validation-rmse:6.32627
[280]	validation-rmse:6.32612
[281]	validation-rmse:6.32554
[282]	validation-rmse:6.32526
[283]	validation-rmse:6.32481
[284]	validation-rmse:6.32442
[285]	validation-rmse:6.32436
[286]	validation-rmse:6.32391
[287]	validation-rmse:6.32376
[288]	validation-rmse:6.32325
[289]	validation-rmse:6.32316
[290]	validation-rmse:6.32300
[291]	validation-rmse:6.32237
[292]	validation-rmse:6.32213
[293]	validation-rmse:6.32161
[294]	validation-rmse:6.32095
[295]	validation-rmse:6.32063
[296]	validation-rmse:6.32054
[297]	validation-rmse:6.32023
[298]	validation-rmse:6.32017
[299]	validation-rmse:6.31966
[300]	validation-rmse:6.31967
[301]	validation-rmse:6.31956
[302]	validation-rmse:6.31982
[303]	vali

[544]	validation-rmse:6.28770
[545]	validation-rmse:6.28765
[546]	validation-rmse:6.28758
[547]	validation-rmse:6.28760
[548]	validation-rmse:6.28756
[549]	validation-rmse:6.28714
[550]	validation-rmse:6.28697
[551]	validation-rmse:6.28678
[552]	validation-rmse:6.28678
[553]	validation-rmse:6.28663
[554]	validation-rmse:6.28622
[555]	validation-rmse:6.28624
[556]	validation-rmse:6.28659
[557]	validation-rmse:6.28693
[558]	validation-rmse:6.28708
[559]	validation-rmse:6.28694
[560]	validation-rmse:6.28687
[561]	validation-rmse:6.28681
[562]	validation-rmse:6.28667
[563]	validation-rmse:6.28678
[564]	validation-rmse:6.28640
[565]	validation-rmse:6.28628
[566]	validation-rmse:6.28636
[567]	validation-rmse:6.28625
[568]	validation-rmse:6.28596
[569]	validation-rmse:6.28587
[570]	validation-rmse:6.28574
[571]	validation-rmse:6.28581
[572]	validation-rmse:6.28587
[573]	validation-rmse:6.28600
[574]	validation-rmse:6.28544
[575]	validation-rmse:6.28531
[576]	validation-rmse:6.28530
[577]	vali

2023/05/24 16:33:26 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/05/24 16:33:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/homebrew/anaconda3/envs/exp-tracking-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [18]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'